In [214]:
import pickle
import numpy as np
import pandas as pd
import scipy
import dill
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Создание быстрой модели

In [215]:
class FastModel(torch.nn.Module):
    def __init__(self, n_observations, n_actions):
        super(FastModel, self).__init__()
        self.layer1 = torch.nn.Linear(n_observations, n_observations*3)
        self.layer2 = torch.nn.Linear(n_observations*3, int(n_observations*1))
        self.layer3 = torch.nn.Linear(int(n_observations*1), n_actions)
        
    def forward(self, x):
        x = torch.nn.functional.tanh(self.layer1(x))
        x = torch.nn.functional.tanh(self.layer2(x))
        return self.layer3(x)

In [216]:
model = FastModel(11, 1)
loss = torch.nn.MultiLabelSoftMarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

# Мокнутый датасет

In [217]:
data_customer_const_1 = torch.tensor([
    [0, 25, 0, 0, 4, 1, 5, 5, 1, 5, 10],
    [0, 25, 0, 0, 4, 1, 12, 5, 1, 5, 10],
    [0, 25, 0, 0, 1, 1, 18, 5, 1, 5, 10],
    [0, 25, 0, 0, 5, 1, 20, 5, 1, 5, 10],
    [0, 25, 0, 0, 2, 1, 4, 5, 1, 5, 10],
    [0, 25, 0, 0, 1, 1, 5, 5, 1, 5, 10],
]).float()

# Загрузка либ

In [218]:
with open('./env_of_customer/Env.pkl', 'rb') as f:
    Env = dill.load(f)

In [219]:
with open('./env_of_customer/get_propability_of_walk_age.pkl', 'rb') as f:
    get_propability_of_walk_age = dill.load(f)

In [220]:
with open('./env_of_customer/get_propability_for_checkout_transport.pkl', 'rb') as f:
    get_propability_for_checkout_transport = dill.load(f)

In [221]:
with open('./env_of_customer/change_distance_by_transport.pkl', 'rb') as f:
    change_distance_by_transport = dill.load(f)

In [222]:
with open('models/raiting_check/raiting_checkout_pred_tree.pkl', 'rb') as f:
    raiting_checkout_pred_tree = dill.load(f)

In [223]:
with open('models/raiting_check/pf.pkl', 'rb') as f:
    pf = dill.load(f)

In [224]:
with open('models/raiting_check/dataset_preprocessor_for_raiting.pkl', 'rb') as f:
    dataset_preprocessor_for_raiting = dill.load(f)

In [225]:
with open('models/raiting_check/std_scaler_for_raiting.pkl', 'rb') as f:
    std_scaler_for_raiting = dill.load(f)

In [226]:
with open('./env_of_customer/from_vector_to_dict.pkl', 'rb') as f:
    from_vector_to_dict = dill.load(f)

# Инициализация среды

In [227]:
env = Env(
    get_propability_for_checkout_transport=get_propability_for_checkout_transport,
    change_distance_by_transport=change_distance_by_transport,
    get_propability_of_walk_age=get_propability_of_walk_age,
    propability_of_checkout_branch_by_raiting=lambda a: raiting_checkout_pred_tree.predict(dataset_preprocessor_for_raiting([a], pf, std_scaler_for_raiting))*100
)

In [228]:
for i in data_customer_const_1:
    print(env.step(from_vector_to_dict(i.numpy())))

0.023266403430070647
0.02313088312692809
0.0002919469489068935
0.04660974459481412
0.00029555194434059304
0.00029537570537138967


# Обучение

In [229]:
epochs = 50
for epoch in range(epochs):
    for x_b in data_customer_const_1:
        outputs = model(x_b)
        s = env.step(from_vector_to_dict(x_b.numpy()))
        # print(s)
        loss_value = loss(outputs, s)
        loss_value.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f'Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}')

Эпоха 1, Значение функции потерь: 0.9155105948448181
Эпоха 2, Значение функции потерь: 0.9136586785316467
Эпоха 3, Значение функции потерь: 0.9118034839630127
Эпоха 4, Значение функции потерь: 0.9099489450454712
Эпоха 5, Значение функции потерь: 0.9080963134765625
Эпоха 6, Значение функции потерь: 0.9062464237213135
Эпоха 7, Значение функции потерь: 0.9043991565704346
Эпоха 8, Значение функции потерь: 0.9025550484657288
Эпоха 9, Значение функции потерь: 0.9007139801979065
Эпоха 10, Значение функции потерь: 0.8988760709762573
Эпоха 11, Значение функции потерь: 0.8970410823822021
Эпоха 12, Значение функции потерь: 0.8952091932296753
Эпоха 13, Значение функции потерь: 0.8933804035186768
Эпоха 14, Значение функции потерь: 0.8915547728538513
Эпоха 15, Значение функции потерь: 0.8897320628166199
Эпоха 16, Значение функции потерь: 0.8879123330116272
Эпоха 17, Значение функции потерь: 0.8860957026481628
Эпоха 18, Значение функции потерь: 0.8842821717262268
Эпоха 19, Значение функции потерь: 0.

Эпоха 44, Значение функции потерь: 0.838168203830719
Эпоха 45, Значение функции потерь: 0.8364344239234924
Эпоха 46, Значение функции потерь: 0.8347035050392151
Эпоха 47, Значение функции потерь: 0.8329755067825317
Эпоха 48, Значение функции потерь: 0.8312504887580872
Эпоха 49, Значение функции потерь: 0.8295283317565918
Эпоха 50, Значение функции потерь: 0.82780921459198


# Создание функции

In [230]:
def model_pred(model: torch.nn.Module, loss, data, from_vector_to_dict: callable, env, epochs = 50):
    for _ in range(epochs):
        for x_b in data[torch.randperm(data_customer_const_1.shape[0])]:
            outputs = model(x_b)
            s = env.step(from_vector_to_dict(x_b.numpy()))
            loss_value = loss(outputs, 2**s)
            loss_value.backward()
            optimizer.step()
            optimizer.zero_grad()
    return model(data).detach().numpy()


In [231]:
model = FastModel(11, 1)
loss = torch.nn.MultiLabelSoftMarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

In [232]:
model_pred(
    model=model,
    loss=loss,
    data=data_customer_const_1,
    from_vector_to_dict=from_vector_to_dict,
    env=env,
)

array([[0.49489063],
       [0.48713627],
       [0.52523243],
       [0.52775073],
       [0.49299124],
       [0.49064422]], dtype=float32)

In [233]:
data_customer_const_1[3]

tensor([ 0., 25.,  0.,  0.,  5.,  1., 20.,  5.,  1.,  5., 10.])

# Save

In [234]:
torch.save(FastModel(11, 1), './NNmodel/tensor.pt')

In [235]:
with open('./NNmodel/model.pkl', 'wb') as f:
    dill.dump(FastModel(11, 1), f)

In [236]:
with open('./NNmodel/model_pred.pkl', 'wb') as f:
    dill.dump(model_pred, f)